In [5]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from transformers import ViTImageProcessor, ViTForImageClassification
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
from torch import nn, optim

/home/wfd/miniconda3/envs/PYTORCH/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.empty_cache() # Clear cache
torch.cuda.reset_max_memory_allocated()  # Reset the max memory allocated counter
torch.cuda.reset_accumulated_memory_stats()  # Reset the accumulated memory stats

/home/wfd/miniconda3/envs/PYTORCH/lib/python3.10/site-packages/torch/cuda/memory.py:271: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [2]:
def config_resnet50():
    global transform, processor, model, model_name
    # Define the transformations
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Initialize the processor and model
    processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
    model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

    # Freeze the first X layers
    X = 0 # Number of layers to freeze
    layer_count = 0
    for param in model.parameters():
        if layer_count < X:
            param.requires_grad = False
        layer_count += 1

    # Get the number of in_features from the current layer
    num_ftrs = model.classifier[1].in_features  

    model_name = "microsoft-resnet50"

    model.classifier[1] = nn.Linear(num_ftrs, 2)  # Replace with a new Linear layer with 2 outputs

In [4]:
# Configure the model
config_resnet50()

# Check what layers are trainable
# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

print(model.classifier)

# Specify the number of epochs
num_epochs = 30
batch_size = 30
learning_rate = 0.0001
dataset_path = "./DataClassification/Split_dataset/"

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=2048, out_features=2, bias=True)
)


In [5]:
train_dataset = datasets.ImageFolder(root=dataset_path + 'Train', transform=transform)
val_dataset = datasets.ImageFolder(root=dataset_path + 'Valid', transform=transform)
test_dataset = datasets.ImageFolder(root=dataset_path + 'Test', transform=transform)


# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Train batches: {len(train_loader)}, Validation batches: {len(val_loader)}, Test batches: {len(test_loader)}")

Train batches: 45, Validation batches: 12, Test batches: 22


In [3]:
def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Check for GPU availability
device = get_device()
print(f"Using device: {device}")

# Transfer the model to GPU
model = model.to(device)

Using device: cuda


In [8]:
# Training and validation loop
for epoch in range(num_epochs):
    # Training
    model.train()
    total_loss = 0
    train_total = 0
    train_correct = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images).logits

        # Compute loss
        loss = loss_function(outputs, labels)
        total_loss += loss.item()

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx+1) % 10 == 0:
            pass
            # print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}, Loss: {loss.item()}')

    # Validation
    model.eval()
    val_total_loss = 0
    val_total = 0
    val_correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            # Transfer images and labels to GPU
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images).logits

            # Compute loss
            val_loss = loss_function(outputs, labels)
            val_total_loss += val_loss.item()

            # Compute accuracy
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / len(train_loader)
    train_accuracy = 100 * train_correct / train_total

    # Calculate average validation loss and accuracy
    avg_val_loss = val_total_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total

    print(f'Epoch {epoch+1}, \tAverage Training Loss: \t{avg_loss}, \tTraining Accuracy: \t{train_accuracy}%')
    print(f'Epoch {epoch+1}, \tAverage Validation Loss: \t{avg_val_loss}, \tValidation Accuracy: \t{val_accuracy}%')

Epoch 1, 	Average Training Loss: 	0.6223981592390272, 	Training Accuracy: 	61.404833836858%
Epoch 1, 	Average Validation Loss: 	0.4762379253904025, 	Validation Accuracy: 	96.62921348314607%
Epoch 2, 	Average Training Loss: 	0.41250076956219145, 	Training Accuracy: 	87.7643504531722%
Epoch 2, 	Average Validation Loss: 	0.22052468887219825, 	Validation Accuracy: 	99.43820224719101%
Epoch 3, 	Average Training Loss: 	0.24101612584458457, 	Training Accuracy: 	94.41087613293051%
Epoch 3, 	Average Validation Loss: 	0.08820887717107932, 	Validation Accuracy: 	99.15730337078652%
Epoch 4, 	Average Training Loss: 	0.10272103266583549, 	Training Accuracy: 	97.88519637462235%
Epoch 4, 	Average Validation Loss: 	0.06164344431211551, 	Validation Accuracy: 	99.15730337078652%
Epoch 5, 	Average Training Loss: 	0.05726743133531676, 	Training Accuracy: 	98.56495468277946%
Epoch 5, 	Average Validation Loss: 	0.07278117071837187, 	Validation Accuracy: 	97.19101123595506%
Epoch 6, 	Average Training Loss: 	0

In [9]:
# Testing loop
model.eval()  # Set the model to evaluation mode
test_total = 0
test_correct = 0

with torch.no_grad():  # No gradients needed for testing, saves memory and computations
    for images, labels in test_loader:
        # Transfer images and labels to GPU (if using)
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images).logits

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)    
        test_correct += (predicted == labels).sum().item()

# Print test accuracy
test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy}%')   

Test Accuracy: 99.0909090909091%


In [12]:
# Save the model
torch.save(model.state_dict(), f'./Models/{model_name}.pth')

# Print the max memory allocated
print(torch.cuda.max_memory_allocated() / 1024 ** 2, 'MB')

3304.171875 MB


In [6]:
# Let's predict on a whole folder of images
import os
from PIL import Image
import numpy as np

device = get_device()

#! We reconfigure the model so that this cell can be run independently
config_resnet50()

# Load model from saved file to make sure we are using the model we just trained
#? Could be skipped but added for clarity
model.load_state_dict(torch.load(f'./Models/{model_name}.pth'))
model.eval()
model.to(device)

# Get the list of images
dir_list = os.listdir('./Classify')

sclerosed = []
images = []

# Get the list of images by dir
for dir in dir_list:
    image_list = os.listdir(f'./Classify/{dir}')

    i = 0
    # Loop through the images
    for image in image_list:
        images.append(image)
        
        # Load the image
        img = Image.open(f'./Classify/{dir}/{image}')

        # Transform the image
        img = transform(img)

        # Add a batch dimension
        img = img.unsqueeze(0)

        # Transfer the image to GPU (if using)
        img = img.to(device)

        # Forward pass
        output = model(img).logits

        # Get the predicted class
        _, predicted = torch.max(output.data, 1)

        i += 1
        if (predicted.item()):
            # Print the prediction
            print(f'Image \t{image[24:-5]} \tof {dir} is predicted to be a sclerosed glomerulus')
            sclerosed.append(image)

print(f"Total sclerosed: {len(sclerosed)}")

# Save the list of sclerosed images as csv
import csv

with open(f'./Predictions/{model_name}.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Loop through the list of images
    for image in images:
        writer.writerow([image, 1 if image in sclerosed else 0])

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Image 	PAS [78656, 12036, 864, 880] 	of C2326935-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [142544, 35640, 832, 888] 	of C2326935-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [74804, 11792, 808, 836] 	of C2326935-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [78112, 11540, 876, 928] 	of C2326935-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [146104, 24548, 844, 1008] 	of C2326935-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [135024, 56940, 880, 916] 	of C2327104-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [133192, 48412, 988, 880] 	of C2327104-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [88196, 20896, 864, 760] 	of C2327104-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [135364, 57740, 632, 636] 	of C2327104-1-A-PAS is predicted to be a sclerosed glomerulus
Image 	PAS [132888, 52828, 828, 868] 	of C2327104-1-A-PAS is predicted to be a sclerosed glomerulus
Ima